# Symulacja 05: Asymilacja Danych (ABC vs 4D-Var)

Notebook realizuje eksperyment asymilacji danych dla modeli wzrostu guza (PDE i ODE). 

**Metody:**
1. **ABC (Approximate Bayesian Computation)** - metoda bez gradientów, oparta na próbkowaniu.
2. **4D-Var (Four-Dimensional Variational Assimilation)** - metoda wariacyjna minimalizująca funkcję kosztu na przestrzeni okna czasowego (trajektorii).

**Cel:** Estymacja parametrów `rho`, `beta`, `K`, `D` na podstawie zaszumionych obserwacji syntetycznych.

In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
from pathlib import Path

# Sprawdzenie dostępności SciPy dla 4D-Var
try:
    from scipy.optimize import minimize
    _SCIPY_AVAILABLE = True
except ImportError:
    _SCIPY_AVAILABLE = False
    print("Warning: SciPy not found. 4D-Var will fail if attempted.")

In [23]:
# ==========================================
# KONFIGURACJA ŚCIEŻEK
# ==========================================

DEFAULT_OUTPUT_DIR = Path('../figures/sim_05')
DEFAULT_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Output directory set to: {DEFAULT_OUTPUT_DIR.resolve()}")

# Podfoldery dla porządku
(DEFAULT_OUTPUT_DIR / "pde").mkdir(exist_ok=True)
(DEFAULT_OUTPUT_DIR / "ode").mkdir(exist_ok=True)
(DEFAULT_OUTPUT_DIR / "gifs").mkdir(exist_ok=True)

Output directory set to: /Users/igorzakrocki/Library/CloudStorage/OneDrive-AkademiaGórniczo-Hutniczaim.StanisławaStaszicawKrakowie/AGH/IDS_9/Informatyka Systemów Złożonych/CS_radiotherapy_project/figures/sim_05


## 1. Zintegrowany Model Fizyczny (PDE & ODE)
Definicja siatki, stałych fizycznych oraz solvera symulacji.

In [24]:
# ---- Global Grid & Physics Constants ----
L = 1.0
N = 101
dx = L / (N - 1)
x = np.linspace(0, L, N)

# 1. Spatial Hypoxia Profile
H_vec = 0.4 + 0.6 * np.exp(-((x - 0.5 * L)/0.5) ** 2)

# 2. Effective/Mean Hypoxia
H_eff = np.mean(H_vec)

# Problem bounds for DA
# Note: D upper bound restricted to 0.0045 for CFL stability
problem = {
    "names": ["rho", "beta", "K", "D"],
    "bounds": [
        [0.1, 0.4],     # rho
        [0.5, 1.5],     # beta
        [0.8, 1.5],     # K
        [0.0001, 0.0045]# D
    ]
}

def dose_rate(t):
    """Calculates dose rate at time t based on fractionation schedule."""
    dose_amp = 4.0
    fraction_duration = 0.2
    course_starts = [5.0, 15.0, 25.0]

    for cs in course_starts:
        for n in range(5):
            t_start = cs + n * 1.0
            t_end = t_start + fraction_duration
            if t_start <= t <= t_end:
                return dose_amp
    return 0.0

def run_simulation(params, model_type="PDE"):
    """
    Runs the simulation for a given set of parameters.
    Returns: (y_hist, t_hist)
    """
    # Unpack parameters
    rho, beta, K, D = params

    T_end = 35.0
    dt = 0.01
    nsteps = int(T_end / dt)

    # Initial conditions
    u = 0.2 * np.ones(N)  # PDE spatial
    U_ode = 0.2           # ODE scalar

    t_hist = np.zeros(nsteps + 1)
    y_hist = np.zeros(nsteps + 1)

    t_hist[0] = 0.0
    # Store initial state
    y_hist[0] = np.mean(u) if model_type == "PDE" else U_ode

    # Time integration loop
    for n in range(1, nsteps + 1):
        t = n * dt
        R_t = dose_rate(t)

        if model_type == "PDE":
            # Explicit Euler with Neumann BCs
            u_xx = np.zeros_like(u)
            u_xx[1:-1] = (u[2:] - 2 * u[1:-1] + u[:-2]) / dx**2
            u_xx[0] = 2 * (u[1] - u[0]) / dx**2
            u_xx[-1] = 2 * (u[-2] - u[-1]) / dx**2

            # Reaction-Diffusion equation
            du_dt = D * u_xx + rho * u * (1 - u / K) - beta * R_t * H_vec * u
            u = u + dt * du_dt
            
            # Safety check
            if not np.all(np.isfinite(u)) or np.max(u) > 1e6:
                return np.full(nsteps + 1, 1e6), t_hist

            u = np.clip(u, 0.0, None) # Physical constraint
            y_hist[n] = np.mean(u)

        elif model_type == "ODE":
            # Surrogate ODE for spatial mean
            dU_dt = rho * U_ode * (1 - U_ode / K) - beta * H_eff * R_t * U_ode
            U_ode = U_ode + dt * dU_dt
            
            if not np.isfinite(U_ode) or U_ode > 1e6:
                 return np.full(nsteps + 1, 1e6), t_hist

            U_ode = max(U_ode, 0.0)
            y_hist[n] = U_ode

        t_hist[n] = t

    return y_hist, t_hist

## 2. Ustawienia Eksperymentu DA i Styl

In [25]:
RNG = np.random.default_rng(1234)

PARAM_BOUNDS = np.array(problem["bounds"])
PARAM_NAMES = problem["names"]
TRUE_PARAMS = np.array([0.25, 1.0, 1.2, 0.0025])  # [rho, beta, K, D]

# Assimilation window
ASSIM_START = 8.0
ASSIM_END = 22.0

TIME_BUDGETS = [
    {"label": "low", "abc_samples": 500, "var_iters": 20},
    {"label": "medium", "abc_samples": 2000, "var_iters": 50},
    {"label": "high", "abc_samples": 8000, "var_iters": 100},
]

# --- Scientific Dark Style ---
def set_style():
    plt.style.use('dark_background')
    
    bg_color = "#252629"
    axes_color = "#1e1e1e"
    grid_color = "#333333"
    text_color = "#e0e0e0"

    plt.rcParams.update({
        "figure.facecolor": bg_color,
        "axes.facecolor": axes_color,
        "savefig.facecolor": bg_color,
        "font.family": "sans-serif",
        "font.size": 11,
        "text.color": text_color,
        "axes.labelcolor": text_color,
        "xtick.color": text_color,
        "ytick.color": text_color,
        "axes.edgecolor": "#444444",
        "grid.color": grid_color,
        "grid.linestyle": "--",
        "grid.alpha": 0.6,
        "legend.facecolor": axes_color,
        "legend.edgecolor": "#444444",
    })

set_style()

COLORS = {
    "truth": "#f0f0f0",
    "obs": "#ffcc00",
    "abc": "#00d4ff",
    "var": "#ff00cc",
    "window": "#334455",
    "hist": "#00d4ff",
    "hist_edge": "#1e1e1e",
    "true_line": "#ff3333"
}

## 3. Algorytmy Asymilacji (ABC i 4D-Var)

In [26]:
def compute_rmse(y_est, y_true, mask):
    diff = y_est[mask] - y_true[mask]
    return float(np.sqrt(np.mean(diff ** 2)))

def generate_truth_and_observations():
    # True PDE trajectory
    y_pde_true, t = run_simulation(TRUE_PARAMS, model_type="PDE")
    # True ODE trajectory
    y_ode_true, t_ode = run_simulation(TRUE_PARAMS, model_type="ODE")
    
    # Observations
    stride = 4
    obs_indices = np.arange(0, len(t), stride)
    t_obs = t[obs_indices]

    noise_std_pde = 0.01
    y_obs_pde = y_pde_true[obs_indices] + RNG.normal(0.0, noise_std_pde, size=len(obs_indices))

    noise_std_ode = 0.05
    y_obs_ode = y_pde_true[obs_indices] + RNG.normal(0.0, noise_std_ode, size=len(obs_indices))

    assim_mask_obs = (t_obs >= ASSIM_START) & (t_obs <= ASSIM_END)
    fwd_mask = t > ASSIM_END
    bwd_mask = t < ASSIM_START

    return (
        {"t": t, "pde": y_pde_true, "ode": y_ode_true},
        {"t_obs": t_obs, "pde": y_obs_pde, "ode": y_obs_ode},
        {"obs_indices": obs_indices, "assim_mask_obs": assim_mask_obs, 
         "fwd_mask": fwd_mask, "bwd_mask": bwd_mask}
    )

# --- ABC Algorithm ---
def abc_da(model_type, y_obs_assim, full_obs_indices, assim_mask_obs, N_samples, seed=None):
    rng = np.random.default_rng(seed)
    n_dim = PARAM_BOUNDS.shape[0]
    assim_obs_indices = full_obs_indices[assim_mask_obs]
    
    params_all = np.empty((N_samples, n_dim))
    distances = np.empty(N_samples)

    for i in range(N_samples):
        theta = rng.uniform(PARAM_BOUNDS[:, 0], PARAM_BOUNDS[:, 1])
        params_all[i] = theta
        y_model, _ = run_simulation(theta, model_type=model_type)
        y_model_assim = y_model[assim_obs_indices]
        distances[i] = compute_rmse(y_model_assim, y_obs_assim, mask=np.ones_like(y_obs_assim, dtype=bool))

    # Handle NaNs in distances by assigning infinity
    distances = np.nan_to_num(distances, nan=np.inf)

    epsilon = float(np.quantile(distances, 0.15))
    accepted = params_all[distances <= epsilon]
    
    if accepted.size > 0:
        theta_mean = accepted.mean(axis=0)
    else:
        theta_mean = params_all[np.argmin(distances)]

    return {"theta_mean": theta_mean, "accepted": accepted}

# --- 4D-Var Algorithm ---
def four_d_var_da(model_type, y_obs_assim, full_obs_indices, assim_mask_obs, x_b, B, sigma_obs, maxiter):
    """
    Strong Constraint 4D-Var: Minimizes cost function over the time trajectory.
    J(theta) = J_b + J_o = (x-xb)T B-1 (x-xb) + Sum[(H(x)_i - y_i)^2 / sigma^2]
    """
    if not _SCIPY_AVAILABLE: raise RuntimeError("SciPy missing")
    assim_obs_indices = full_obs_indices[assim_mask_obs]
    B_inv = np.linalg.inv(B)
    sigma2 = sigma_obs ** 2

    def cost(theta):
        # 1. Background term (Prior)
        diff_b = theta - x_b
        J_b = diff_b @ B_inv @ diff_b
        
        # 2. Observation term (trajectory misfit over time window)
        y_model, _ = run_simulation(theta, model_type=model_type)
        
        # Check for unstable simulation
        if np.max(y_model) > 1e5: 
            return 1e12

        diff_o = y_model[assim_obs_indices] - y_obs_assim
        J_o = np.sum((diff_o ** 2) / sigma2)
        
        return 0.5 * (J_b + J_o)

    res = minimize(cost, x0=x_b, method="L-BFGS-B", bounds=PARAM_BOUNDS, options={"maxiter": maxiter, "disp": False})
    return res.x, res

## 4. Funkcje Wizualizacji
Wszystkie wykresy zapisywane są do `../figures/sim_05/`.

In [27]:
def plot_time_series(model_type, t, y_true, t_obs, y_obs,
                     y_pred_abc, y_pred_var, label_suffix, out_dir):
    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(t, y_true, color=COLORS["truth"], alpha=0.8, 
            label="Truth" if model_type == "PDE" else "Truth (ODE)", lw=2.5, zorder=1)

    ax.scatter(t_obs, y_obs, color=COLORS["obs"], label="Observations", 
               s=35, alpha=0.9, edgecolors="#121212", linewidth=0.5, zorder=3)

    if y_pred_abc is not None:
        ax.plot(t, y_pred_abc, color=COLORS["abc"], linestyle="--", 
                label=f"ABC {label_suffix}", lw=2, zorder=2)
    if y_pred_var is not None:
        ax.plot(t, y_pred_var, color=COLORS["var"], linestyle="-.", 
                label=f"4D-Var {label_suffix}", lw=2, zorder=2)

    ax.axvspan(ASSIM_START, ASSIM_END, color=COLORS["window"], alpha=0.3, 
               label="Assimilation Window", zorder=0)

    ax.set_xlabel("Time", fontweight='bold')
    ax.set_ylabel("Mean Tumour Density", fontweight='bold')
    ax.set_title(f"{model_type} Model: Truth vs DA Predictions", pad=15)
    ax.grid(True)
    ax.legend(loc="upper right", framealpha=0.9)

    fname = out_dir / f"{model_type.lower()}_timeseries_{label_suffix.replace(' ', '_')}.png"
    fig.savefig(fname, bbox_inches="tight", dpi=150)
    plt.close(fig)
    # plt.show() # Uncomment if you want inline plots


def plot_abc_posteriors(model_type, posterior_samples, out_dir, title_suffix):
    if posterior_samples.size == 0:
        return

    n_params = posterior_samples.shape[1]
    n_rows = 2
    n_cols = 2

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 8))
    axes = axes.flatten()

    for i in range(n_params):
        ax = axes[i]
        ax.hist(posterior_samples[:, i], bins=25, density=True, 
                color=COLORS["hist"], alpha=0.7, 
                edgecolor=COLORS["hist_edge"], linewidth=1)
        ax.axvline(TRUE_PARAMS[i], color=COLORS["true_line"], linestyle="--", 
                   linewidth=2, label="True Value")
        ax.set_title(PARAM_NAMES[i], fontweight='bold', fontsize=12)
        if i == 0:
            ax.legend(loc='upper right', framealpha=0.8, fontsize=9)
        ax.grid(True, axis='y', alpha=0.3)

    fig.suptitle(f"{model_type} ABC Posterior Distribution – {title_suffix}", 
                 fontsize=16, y=0.98)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    fname = out_dir / f"{model_type.lower()}_abc_posterior_{title_suffix.replace(' ', '_')}.png"
    fig.savefig(fname, dpi=120) 
    plt.close(fig)


def plot_rmse_budgets(model_type, budget_labels, rmse_fwd_abc, rmse_bwd_abc,
                      rmse_fwd_var, rmse_bwd_var, out_dir):
    x = np.arange(len(budget_labels))
    width = 0.18
    fig, ax = plt.subplots(figsize=(12, 6))
    
    c_abc_fwd, c_abc_bwd = "#00d4ff", "#0088aa"
    c_var_fwd, c_var_bwd = "#ff00cc", "#aa0088"
    bar_kw = dict(edgecolor="#121212", linewidth=0.8, alpha=0.9)

    ax.bar(x - 1.5 * width, rmse_fwd_abc, width, label="ABC Forward", color=c_abc_fwd, **bar_kw)
    ax.bar(x - 0.5 * width, rmse_bwd_abc, width, label="ABC Backward", color=c_abc_bwd, **bar_kw)
    ax.bar(x + 0.5 * width, rmse_fwd_var, width, label="4D-Var Forward", color=c_var_fwd, **bar_kw)
    ax.bar(x + 1.5 * width, rmse_bwd_var, width, label="4D-Var Backward", color=c_var_bwd, **bar_kw)

    ax.set_xticks(x)
    ax.set_xticklabels([l.upper() for l in budget_labels], fontweight='bold')
    ax.set_xlabel("Computational Budget", fontsize=12)
    ax.set_ylabel("RMSE", fontsize=12)
    ax.set_title(f"{model_type} Model: Forward vs Backward RMSE", fontsize=14, pad=15)
    ax.legend(ncol=2, loc="upper right", framealpha=0.9)
    ax.grid(True, axis="y", alpha=0.4)

    fname = out_dir / f"{model_type.lower()}_rmse_vs_budget.png"
    fig.savefig(fname, bbox_inches="tight", dpi=150)
    plt.close(fig)


def make_abc_gif_pde(abc_results_per_budget):
    img_paths = []
    pde_dir = DEFAULT_OUTPUT_DIR / "pde"
    for label, res in abc_results_per_budget:
        samples = res["posterior"]
        if samples.size == 0: continue
        title_suffix = f"budget_{label}"
        plot_abc_posteriors("PDE", samples, pde_dir, title_suffix)
        fname = pde_dir / f"pde_abc_posterior_{title_suffix.replace(' ', '_')}.png"
        img_paths.append(fname)

    if img_paths:
        images = [imageio.imread(p) for p in img_paths]
        gif_path = DEFAULT_OUTPUT_DIR / "gifs" / "pde_abc_posteriors.gif"
        imageio.mimsave(gif_path, images, duration=1000, loop=0)

## 5. Główna pętla programu

In [28]:
def run_da_for_model(model_type, truth, obs, indices):
    full_t = truth["t"]
    full_true = truth["pde"] if model_type == "PDE" else truth["ode"]
    y_obs_all = obs["pde"] if model_type == "PDE" else obs["ode"]
    
    obs_indices = indices["obs_indices"]
    assim_mask_obs = indices["assim_mask_obs"]
    y_obs_assim = y_obs_all[assim_mask_obs]
    t_obs_assim = obs["t_obs"][assim_mask_obs]

    results_abc, results_var = [], []
    
    # 4D-Var Background Setup
    perturb = np.array([0.05, -0.1, 0.1, -0.001])
    x_b = np.clip(TRUE_PARAMS + perturb, PARAM_BOUNDS[:, 0], PARAM_BOUNDS[:, 1])
    B = np.diag([0.05 ** 2, 0.2 ** 2, 0.2 ** 2, (0.002 ** 2)])

    for budget in TIME_BUDGETS:
        print(f"[{model_type} | {budget['label']}] Running...")
        
        # --- ABC ---
        abc_res = abc_da(model_type, y_obs_assim, obs_indices, assim_mask_obs, budget['abc_samples'], seed=42)
        y_pred_abc, _ = run_simulation(abc_res["theta_mean"], model_type=model_type)
        
        results_abc.append({
            "label": budget['label'],
            "posterior": abc_res["accepted"],
            "theta": abc_res["theta_mean"],
            "rmse_fwd": compute_rmse(y_pred_abc, full_true, indices["fwd_mask"]),
            "rmse_bwd": compute_rmse(y_pred_abc, full_true, indices["bwd_mask"])
        })

        # --- 4D-Var ---
        if _SCIPY_AVAILABLE:
            theta_opt, _ = four_d_var_da(model_type, y_obs_assim, obs_indices, assim_mask_obs, x_b, B, 0.03, budget['var_iters'])
            y_pred_var, _ = run_simulation(theta_opt, model_type=model_type)
            results_var.append({
                "label": budget['label'],
                "theta": theta_opt,
                "rmse_fwd": compute_rmse(y_pred_var, full_true, indices["fwd_mask"]),
                "rmse_bwd": compute_rmse(y_pred_var, full_true, indices["bwd_mask"])
            })
        else:
             results_var.append({"label": budget['label'], "theta": x_b, "rmse_fwd": 0, "rmse_bwd": 0})

    # Best predictions
    y_pred_abc_best, _ = run_simulation(results_abc[-1]["theta"], model_type=model_type)
    y_pred_var_best = run_simulation(results_var[-1]["theta"], model_type=model_type)[0] if _SCIPY_AVAILABLE else None

    return {
        "abc": results_abc, "var": results_var,
        "predictions": {
            "abc_best": y_pred_abc_best, "var_best": y_pred_var_best,
            "t": full_t, "y_true": full_true, 
            "t_obs": t_obs_assim, "y_obs": y_obs_assim
        }
    }

# === Uruchomienie Eksperymentu ===
print("Starting Standalone DA Experiment (Stable Version)...")
truth, obs, indices = generate_truth_and_observations()

for model_type in ["PDE", "ODE"]:
    print(f"\n--- Processing {model_type} Model ---")
    res = run_da_for_model(model_type, truth, obs, indices)
    
    # Plotting
    p = res["predictions"]
    budget_labels = [b["label"] for b in TIME_BUDGETS]
    out_sub = DEFAULT_OUTPUT_DIR / model_type.lower()
    
    # 1. Time Series
    plot_time_series(model_type, p["t"], p["y_true"], p["t_obs"], p["y_obs"],
                     p["abc_best"], p["var_best"], f"budget={budget_labels[-1]}", out_sub)
    
    # 2. RMSE Bar Chart
    plot_rmse_budgets(model_type, budget_labels,
                      [r["rmse_fwd"] for r in res["abc"]], [r["rmse_bwd"] for r in res["abc"]],
                      [r["rmse_fwd"] for r in res["var"]], [r["rmse_bwd"] for r in res["var"]],
                      out_sub)
    
    # 3. Posteriors
    plot_abc_posteriors(model_type, res["abc"][-1]["posterior"], out_sub, f"budget={budget_labels[-1]}")
    
    # 4. GIF (only for PDE)
    if model_type == "PDE":
        print("Generating PDE GIF...")
        make_abc_gif_pde([(r["label"], r) for r in res["abc"]])

print(f"\nDone. Results saved in '{DEFAULT_OUTPUT_DIR}/'.")

Starting Standalone DA Experiment (Stable Version)...

--- Processing PDE Model ---
[PDE | low] Running...
[PDE | medium] Running...
[PDE | high] Running...
Generating PDE GIF...

--- Processing ODE Model ---
[ODE | low] Running...
[ODE | medium] Running...
[ODE | high] Running...

Done. Results saved in '../figures/sim_05/'.
